# import libary

In [48]:
import os, sys, logging, pandas as pd, numpy as np
from tqdm import tqdm
import re

sys.path.insert(0, '../')



import ast
import util
from importlib import reload
reload(util)
args = util.parser.parse_args([])



# import librosa
# from IPython.display import Audio

from pandas import json_normalize
import torch
from collections import defaultdict

from multiprocessing import Pool


# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions

# os.environ['HTTP_PROXY'] = 'socks5h://127.0.0.1:6000'
# os.environ['HTTPS_PROXY'] = 'socks5h://127.0.0.1:6000'


pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

PROJECT="mls25"
ADLSRV="root@connect.westc.gpuhub.com"
ADLPORT="39023"
ADLPASS="lpXgdzbPVom0"
WORKDIR = os.path.abspath(f'/var/opt/git/github/compete/{PROJECT}')
os.chdir(os.path.join(WORKDIR, PROJECT))

# sync code

In [132]:
!sshpass -p "{ADLPASS}" rsync -rza --progress --exclude {PROJECT}/sft*.py --exclude {PROJECT}/train*.py --exclude {PROJECT}/preprocess*.py \
--exclude {PROJECT}/asr*.py  --exclude {PROJECT}/e2e*.py  --exclude {PROJECT}/eval*.py \
--exclude {PROJECT}/notebooks -e "ssh -p {ADLPORT}" \
--exclude {PROJECT}/notes.txt --exclude {PROJECT}/*.ipynb --exclude {PROJECT}/README.md --exclude {PROJECT}/data \
../../{PROJECT} {ADLSRV}:/tmp/
!sshpass -p "{ADLPASS}" ssh -p {ADLPORT} {ADLSRV} "mkdir -p /root/autodl-tmp/mydata/{PROJECT} && cd /tmp/{PROJECT} && ln -s /root/autodl-tmp/mydata data"
!sshpass -p "{ADLPASS}" ssh -p {ADLPORT} {ADLSRV} "mkdir -p /root/autodl-tmp/mydata/cache/huggingface/hub && mkdir -p /root/autodl-tmp/mydata/logs"

sending incremental file list
mls25/nohup.out
      9,068,727 100%  785.98MB/s    0:00:00 (xfr#1, ir-chk=1020/1026)
ln: failed to create symbolic link 'data/mydata': File exists


# sync data

## upload

In [10]:
!sshpass -p "{ADLPASS}" rsync -rza --progress -e "ssh -p $ADLPORT" ../data/mls25/*.zip $ADLSRV:/root/autodl-tmp/mydata/mls25





sending incremental file list
alg_2025_audios_final.zip
  2,738,591,241 100%    3.20MB/s    0:13:36 (xfr#1, to-chk=1/2)
text_data.zip
      2,410,227 100%    1.75MB/s    0:00:01 (xfr#2, to-chk=0/2)


In [110]:
!sshpass -p "{ADLPASS}" rsync -rza --progress -e "ssh -p $ADLPORT" ../data/alt.csv $ADLSRV:/root/autodl-tmp/mydata

sending incremental file list
alt.csv
     29,237,812 100%   39.77MB/s    0:00:00 (xfr#1, to-chk=0/1)


In [51]:
#!sshpass -p "{ADLPASS}" rsync -rza --progress -e "ssh -p $ADLPORT" ../data/k2-1.24.4.dev20250425+cuda12.8.torch2.7.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl $ADLSRV:/root/autodl-tmp/mydata/

!sshpass -p "{ADLPASS}" rsync -rza --progress -e "ssh -p $ADLPORT" ../data/k2-1.24.4.dev20250425+cuda12.6.torch2.7.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl $ADLSRV:/root/autodl-tmp/mydata/


sending incremental file list
k2-1.24.4.dev20250425+cuda12.6.torch2.7.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl
    111,976,624 100%    7.14MB/s    0:00:14 (xfr#1, to-chk=0/1)


## datasets

In [ ]:
fpath="/mnt/usbdisk1/data/cache/huggingface/datasets/open-r1___open_r1-math-220k"
!sshpass -p "{ADLPASS}" rsync -rza --progress -e "ssh -p $ADLPORT" "{fpath}" $ADLSRV:/root/autodl-tmp/mydata/cache/huggingface/hub

## download

In [144]:
!sshpass -p "{ADLPASS}" rsync -rza --progress -e "ssh -p $ADLPORT"  $ADLSRV:/root/autodl-tmp/mydata/Trans_qw_8b_a03_KF0 ../data

receiving incremental file list
Trans_qw_8b_a03_KF0/
Trans_qw_8b_a03_KF0/args.json
          3,174 100%    3.03MB/s    0:00:00 (xfr#1, to-chk=14/16)
Trans_qw_8b_a03_KF0/events.out.tfevents.1749521809.autodl-container-dc0b44b26f-25e70951.13340.0
          9,926 100%    9.47MB/s    0:00:00 (xfr#2, to-chk=13/16)
Trans_qw_8b_a03_KF0/checkpoint-1006/
Trans_qw_8b_a03_KF0/checkpoint-1006/README.md
          5,107 100%    4.87MB/s    0:00:00 (xfr#3, to-chk=11/16)
Trans_qw_8b_a03_KF0/checkpoint-1006/adapter_config.json
            865 100%  844.73kB/s    0:00:00 (xfr#4, to-chk=10/16)
Trans_qw_8b_a03_KF0/checkpoint-1006/adapter_model.safetensors
    349,243,752 100%   10.14MB/s    0:00:32 (xfr#5, to-chk=9/16)
Trans_qw_8b_a03_KF0/checkpoint-1006/added_tokens.json
            707 100%    1.35kB/s    0:00:00 (xfr#6, to-chk=8/16)
Trans_qw_8b_a03_KF0/checkpoint-1006/chat_template.jinja
          4,761 100%    9.08kB/s    0:00:00 (xfr#7, to-chk=7/16)
Trans_qw_8b_a03_KF0/checkpoint-1006/merges.txt
    

In [63]:
!sshpass -p "{ADLPASS}" rsync -rza --progress -e "ssh -p $ADLPORT"  $ADLSRV:/root/autodl-tmp/mydata/logs/log20250602091734.log ../data/logs


receiving incremental file list
log20250602091734.log
          5,359 100%    5.11MB/s    0:00:00 (xfr#1, to-chk=0/1)


# sync models

In [48]:
fpath="/mnt/usbdisk1/data/cache/huggingface/hub/models--unsloth--deepseek-r1-distill-qwen-7b-unsloth-bnb-4bit"
!sshpass -p "{ADLPASS}" rsync -rza --progress -e "ssh -p $ADLPORT" "{fpath}" $ADLSRV:/root/autodl-tmp/mydata/cache/huggingface/hub

X11 forwarding request failed on channel 0
sending incremental file list
rsync: link_stat "/mnt/usbdisk1/data/cache/huggingface/hub/models--unsloth--deepseek-r1-distill-qwen-7b-unsloth-bnb-4bit" failed: No such file or directory (2)
rsync error: some files/attrs were not transferred (see previous errors) (code 23) at main.c(1205) [sender=3.1.3]


# program

## trans

### qwen32b

In [47]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_32b_a01"

DS="mls25"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/hfmodels/unsloth/Qwen3-32B-unsloth-bnb-4bit"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 3e-5 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 8 -gas 4 -vbs 8 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-use_unsloth -use_4bit -use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj 





../data/logs/log20250606183227.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


### qw14b

In [94]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_14b_a01"

DS="mls25"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/hfmodels/unsloth/Qwen3-14B-unsloth-bnb-4bit"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 6e-5 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 8 -gas 4 -vbs 8 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-use_unsloth -use_4bit -use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj 





../data/logs/log20250603175429.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


### qw8b

In [142]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_8b_a11"

DS="mls25"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/Trans_qw_8b_a08_KF0"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"
#0.6363314390182495



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 512  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj





../data/logs/log20250611134102.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [139]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_8b_a10"

DS="alt"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/hfmodels/unsloth/Qwen3-8B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"

#adamw_8bit

!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 512  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt -optim apollo_adamw  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj





../data/logs/log20250611100536.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [124]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_8b_a08"

DS="alt"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/hfmodels/unsloth/Qwen3-8B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"
#0.6363314390182495



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 512  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj





../data/logs/log20250611090526.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [ ]:
#../data/logs/log20250611071906.log

In [107]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_8b_a07"

DS="mls25"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/Trans_qw_8b_a03_KF0"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"
# 0.5760611295700073

!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 512  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj  





../data/logs/log20250610204741.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [77]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_8b_a05"

DS="mls25"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/Trans_qw_8b_a03_KF0"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 512  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj 





../data/logs/log20250610121538.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [76]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_8b_a04"

DS="mls25"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/Trans_qw_8b_a03_KF0"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 512  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj 





../data/logs/log20250610114717.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证
bash: line 1: 26723 Killed                  PYTHONPATH=../ HF_HOME=../data/cache/huggingface python -u - -model_name Trans_qw_8b_a04 -ds mls25 -ds_cls Dataset -val_ds_cls Dataset -test_ds_cls IterDataset -kn 5 -kfids 0 -backbone ../data/Trans_qw_8b_a03_KF0 -data_seed 98740230 -seed 98402392 -eval_delay 0 -epochs 1 -max_seq_len 512 -do_train -n_dl_worker 8 -es 1 -verbose 64 -save_strategy epoch -groupfy -group_col name -n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16 -ppt trans_ppt -use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj > ../data/logs/log20250610114717.log 2>&1


In [74]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_8b_a03"

DS="alt"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/hfmodels/unsloth/Qwen3-8B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"

#0.6954387426376343

!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 512  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj 





../data/logs/log20250610101600.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [73]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_8b_a02"

DS="mls25"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/hfmodels/unsloth/Qwen3-8B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 512  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj 





../data/logs/log20250610085649.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [88]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_8b_a01"

DS="mls25"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/hfmodels/unsloth/Qwen3-8B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 8 -es 1 -verbose 64  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 8 -gas 4 -vbs 8 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-use_unsloth -use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj 





../data/logs/log20250603093418.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


###  qw4b

In [78]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="Trans_qw_1d7b_a01"

DS="mls25"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="IterDataset"
RESTORE=""
BACKBONE= "../data/hfmodels/unsloth/Qwen3-1.7B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 8 -es 1 -verbose 32  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 1e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 8 -gas 4 -vbs 1 -torch_dtype bfloat16 -mixed_precision bf16  -ppt trans_ppt  \
-lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj 





../data/logs/log20250602122447.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证
Connection to connect.westc.gpuhub.com closed by remote host.


In [24]:
ADLPASS,ADLPORT,ADLSRV

('5HSsqYnCdwoa', '42516', 'root@connect.westc.gpuhub.com')

## STT

### qw2audio7b

In [50]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="STT_qw_7b_a04"

DS="mls25"
DSCLS="STTDataset"
VDSCLS="STTDataset"
TDSCLS="STTDataset"
RESTORE=""
BACKBONE="Qwen/Qwen2-Audio-7B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface TRANSFORMERS_OFFLINE=1 python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 2   -max_seq_len 256  -do_train  -n_dl_worker 24 -es 1 -verbose 32  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 2e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -mixed_precision bf16  -ppt qw2audio_sttppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj  





../data/logs/log20250607060054.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [43]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="STT_qw_7b_a03"

DS="mls25"
DSCLS="STTDataset"
VDSCLS="STTDataset"
TDSCLS="STTDataset"
RESTORE=""
BACKBONE="Qwen/Qwen2-Audio-7B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface TRANSFORMERS_OFFLINE=1 python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 24 -es 1 -verbose 32  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 2e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -mixed_precision bf16  -ppt qw2audio_sttppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj  





../data/logs/log20250606085847.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [42]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="STT_qw_7b_a02"

DS="mls25"
DSCLS="STTDataset"
VDSCLS="STTDataset"
TDSCLS="STTDataset"
RESTORE=""
BACKBONE="Qwen/Qwen2-Audio-7B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface TRANSFORMERS_OFFLINE=1 python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 24 -es 1 -verbose 32  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 2e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -mixed_precision bf16  -ppt qw2audio_asrppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj  





../data/logs/log20250606060720.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [39]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="STT_qw_7b_a01"

DS="mls25"
DSCLS="STTDataset"
VDSCLS="STTDataset"
TDSCLS="STTDataset"
RESTORE=""
BACKBONE="Qwen/Qwen2-Audio-7B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface TRANSFORMERS_OFFLINE=1 python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 24 -es 1 -verbose 32  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 2e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -mixed_precision bf16  -ppt qw2audio_asrppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj  





../data/logs/log20250605164952.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


## ASRLM

### qw2audio7b

In [36]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="ASRLM_qw_7b_a02"

DS="mls25"
DSCLS="ASRLMDataset"
VDSCLS="ASRLMDataset"
TDSCLS="ASRLMDataset"
RESTORE=""
BACKBONE="Qwen/Qwen2-Audio-7B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface TRANSFORMERS_OFFLINE=1 python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 24 -es 1 -verbose 32  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 2e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 1 -gas 32 -vbs 2 -torch_dtype bfloat16 -mixed_precision bf16  -ppt qw2audio_asrppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj  





../data/logs/log20250605112657.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [35]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="ASRLM_qw_7b_a02"

DS="mls25"
DSCLS="ASRLMDataset"
VDSCLS="ASRLMDataset"
TDSCLS="ASRLMDataset"
RESTORE=""
BACKBONE="Qwen/Qwen2-Audio-7B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 24 -es 1 -verbose 32  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 2e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt qw2audio_asrppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj  





../data/logs/log20250605110702.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [34]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="ASRLM_qw_7b_a02"

DS="mls25"
DSCLS="ASRLMDataset"
VDSCLS="ASRLMDataset"
TDSCLS="ASRLMDataset"
RESTORE=""
BACKBONE="Qwen/Qwen2-Audio-7B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 16 -es 1 -verbose 32  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 2e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16  -ppt qw2audio_asrppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj  





../data/logs/log20250605105955.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证
bash: line 1:  2454 Killed                  PYTHONPATH=../ HF_HOME=../data/cache/huggingface python -u - -model_name ASRLM_qw_7b_a02 -ds mls25 -ds_cls ASRLMDataset -val_ds_cls ASRLMDataset -test_ds_cls ASRLMDataset -kn 5 -kfids 0 -backbone Qwen/Qwen2-Audio-7B -data_seed 98740230 -seed 98402392 -eval_delay 0 -epochs 1 -max_seq_len 256 -do_train -n_dl_worker 16 -es 1 -verbose 32 -save_strategy epoch -groupfy -group_col name -n_keep_save 1 -lr 2e-4 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 2 -gas 16 -vbs 4 -torch_dtype bfloat16 -mixed_precision bf16 -ppt qw2audio_asrppt -use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj > ../data/logs/log20250605105955.log 2>&1


In [28]:



import util
LOGDIR=f"../data/logs"
LOGFILE=f"{LOGDIR}/log{util.timestamp()}.log"
print(LOGFILE)

MODELNAME="ASRLM_qw_7b_a01"

DS="mls25"
DSCLS="ASRLMDataset"
VDSCLS="ASRLMDataset"
TDSCLS="ASRLMDataset"
RESTORE=""
BACKBONE="Qwen/Qwen2-Audio-7B"



DATASEED=98740230
SEED=98402392
KN=5
KFIDS="0"



!sshpass -p "{ADLPASS}" ssh -p "{ADLPORT}" {ADLSRV} ". /root/miniconda3/etc/profile.d/conda.sh && conda activate ptm  && source /etc/network_turbo && cd /tmp/{PROJECT}/{PROJECT}  \
&& PYTHONPATH=../ HF_HOME=../data/cache/huggingface  python -u >{LOGFILE} 2>&1" \
<sft.py - -model_name {MODELNAME} -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1   -max_seq_len 256  -do_train  -n_dl_worker 16 -es 1 -verbose 32  -save_strategy epoch  -groupfy -group_col name \
-n_keep_save 1 -lr 4e-5 -lr_warmup_ratio 0.01 -weight_decay 0.01 -bs 4 -gas 8 -vbs 8 -torch_dtype bfloat16 -mixed_precision bf16  -ppt qw2audio_asrppt  \
-use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules q_proj o_proj gate_proj up_proj down_proj k_proj v_proj  





../data/logs/log20250605070649.log
设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [ ]:
# 5090
#{'loss': 9.4434, 'grad_norm': 0.42157796025276184, 'learning_rate': 0.00019739663093415008, 'elapsed': 361.1791639328003, 'epoch': 0.02425965411040038}
#{'loss': 5.932, 'grad_norm': 0.12165558338165283, 'learning_rate': 0.00019249617151607964, 'elapsed': 722.6248219013214, 'epoch': 0.04851930822080076}

#bs 2
{'loss': 9.5706, 'grad_norm': 0.2936970889568329, 'learning_rate': 0.00019739663093415008, 'elapsed': 194.73228240013123, 'epoch': 0.02425965411040038}

# bs 2 no 4bit
{'loss': 9.7295, 'grad_norm': 0.4276919662952423, 'learning_rate': 0.00019739663093415008, 'elapsed': 150.8063714504242, 'epoch': 0.02425965411040038}
{'loss': 6.0361, 'grad_norm': 0.11052064597606659, 'learning_rate': 0.00019249617151607964, 'elapsed': 297.9333517551422, 'epoch': 0.04851930822080076}
{'loss': 6.0627, 'grad_norm': 0.08779320120811462, 'learning_rate': 0.0001875957120980092, 'elapsed': 445.1427390575409, 'epoch': 0.07277896233120114}

#bs 1 no lora oom

# donwload transformer models

In [2]:


from huggingface_hub import snapshot_download
MODELID= 'HuggingFaceTB/SmolLM-135M-Instruct'
snapshot_download(repo_id=MODELID)


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/e1/3c/e13ccf2aa0f112590e98f18678477d4cf050e83ad6f9f210df7cd919b0a92d11/c1464c9ebccc6fb497aa4ccc8d74d29b33f9a9eddc4638a4575719357f532f2d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1727951998&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNzk1MTk5OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2UxLzNjL2UxM2NjZjJhYTBmMTEyNTkwZTk4ZjE4Njc4NDc3ZDRjZjA1MGU4M2FkNmY5ZjIxMGRmN2NkOTE5YjBhOTJkMTEvYzE0NjRjOWViY2NjNmZiNDk3YWE0Y2NjOGQ3NGQyOWIzM2Y5YTllZGRjNDYzOGE0NTc1NzE5MzU3ZjUzMmYyZD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=NRNZSUBwCXCzOHlpHO6y84wai4IP4HMuMZlebuMKPOiqKbPo20hCLxXCaWQac8C3wPsMXu6RRE6yYifRMnXk77SQ656oZR87glG435xUDg3uSsQcVCL8YkaTZzanSHNB9ZcWfUeDm0MTqXLkGdwZK1yfoBtuGisIxHLPzcyRWHt5UPYuQLgE-9e3ICncimwlPpxhG-BO7srYf2QJv3CUslVDv%7Ej5arb0-nFZRqI9JJcn5INCiiJpOSpEj14qaAHAYn

model.safetensors:  39%|###8      | 105M/269M [00:00<?, ?B/s]

'/mnt/usbdisk1/data/cache/huggingface/hub/models--HuggingFaceTB--SmolLM-135M-Instruct/snapshots/fcc320f490e08fdb4b99d935b2c58d40bf35b0d0'

# transformer models

In [32]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM-135M")

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

KeyboardInterrupt: 

# dataset

In [6]:
import dataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")
#tokenizer = AutoTokenizer.from_pretrained("unsloth/Qwen2-1.5B-bnb-4bit")
args = util.parser.parse_args([])
args.ds_cls = 'Dataset'
args.val_ds_cls = 'Dataset'

args.batch_size = 2
args.val_batch_size = 2
args.n_dl_worker = 1

df = util.load_data(args)
ds = dataset.gen_ds(args, 'train', df, tokenizer=tokenizer)


/home/tom/anaconda3/envs/yaa/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/tom/anaconda3/envs/yaa/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
rsts = []
ds_iter = iter(ds)
for i in tqdm(range(len(ds))):
    batch = next(ds_iter)
    # input_ids = [id for id in batch['input_ids'] if id!=151648]
    text = tokenizer.decode(batch['input_ids'])
   
    rsts.append([batch['uid'], text])
df = pd.DataFrame(rsts, columns=["uid",  "text"])
df

100%|████████████████| 4000/4000 [00:01<00:00, 2492.58it/s]


,uid,text
0,aaaf,"[CLS] V (XX XX) shot himself in a motor vehicle.The V's mother called law enforcement, stated the V had made suicidal comments, and she reported the V as missing and suicidal with a firearm.The V was located in a vehicle in a retail parking lot. When law enforcement approached the vehicle the V shot himself.There are no other circumstances.[SEP]"
1,aaby,[CLS] V was XXXX. V was found in the basement of his residence hanging from a strap around the neck in an apparent suicide. V's roommates stated that they did not know of any disorders or medications that V would have taken and no indication of suicidal ideations. EMS was on the scene and pronounced V deceased. No further information regarding circumstance.[SEP]
2,aacl,"[CLS] V was XXXX. V was found in his residence unresponsive as the result of a GSW to the head that was suicidal in nature. V was transported by EMS to the hospital where he died. V's medical history was not included. According to this report, V called his mother from school and asked her to bring him something to eat. When she got to the school she saw V was very upset, and she decided to remove him from school and take him home. She reported that V was very upset lately. V's mother went on to say that V's brother was killed about 11 months prior (could not locate brother's information; unable to link) and V had not been dealing with it very well. V's mother had to go to work, so she called V's brother to tell him that V was upset and acting out and that she wanted him to come over and try to calm V down. V's brother responded to the residence and found V holding a ""machine gun."" V and his brother reportedly argued about choosing a better lifestyle and staying away from gangs and ..."
3,aacn,"[CLS] The victim, a XX XX who had recently returned to his village and was residing with his mother (no age given), his step sister, 38, and her minor son and daughter received a call from his girlfriend (no age given) who expressed to him that she missed him. The following day in the evening hours the victim called the girlfriend and she indicated she did not want anything to do with the victim. While on the phone the victim became upset which prompted the mother to take the phone and tell the girlfriend she should have left the victim alone. The victim then became upset and exited the residence. The step-sisters minor son looked out the window and noticed the victim had a firearm and notified the family. The mother exited the residence and approached the victim and demanded he turn over the rifle which he did. The mother then placed the firearm that she believed to be unloaded back into the arctic entry of the residence. The victim then returned to the residence and tried to call..."
4,aadb,"[CLS] XX XX V found deceased at home by his grandparents, hanging from a basketball hoop in his basement family room, which the decedent used as a bedroom of his grandparents house. According to LE, a check of V's cell phone revealed that V had made suicidal statements by phone earlier. In the text message V sent to his girlfriend, he had complained that he hated his life, and she replied that he might consider doing something more about it than whining. Another text message from V to a friend, he stated that he was going to hang himself.[SEP]"
...,...,...
3995,fhri,"[CLS] The victim was a XX XX who was discovered at home with intentional, self-inflicted cuts to his wrists. There is no mention of EMS arriving/evidence of medical intervention. The weapon involved was an unspecified knife/cutting instrument. According to reports, the victim had a history of suicide attempts. There is no mention of a mental health diagnosis, depressed mood, alcohol/substance abuse, or if a note was found. No further details are provided.[SEP]"
3996,fhrn,"[CLS] The V is a XX XX. The cause of death is acute sodium nitrite intoxication and the manner of death is suicide. LE was called to V's apartment by a roommate and a XX friend aroun

In [18]:
df.text.iloc[0][-200:]

'_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT><IMG_CONTEXT></img>output[[707000707][707000707][770000770][707000707][707000707][770000770][707707000][707707000][770770000]]'

In [31]:
df[df.text.str.startswith('\npython')]

,id,task,text
2,2,007bbfb7,"\npython\nfrom utils import *\nfrom dsl import *\nfrom random import seed as set_seed\nset_seed(780605261)\ncols = interval(1, 10, 1)\nh = unifint(diff_lb, diff_ub, (2, 5))#removeme\nh = 2\nw = unifint(diff_lb, diff_ub, (2, 5))#removeme\nw = 5\nc = canvas(0, (h, w))\nnumcd = unifint(diff_lb, diff_ub, (0, (h * w) // 2))#removeme\nnumcd = 2\nnumc = choice((numcd, h * w - numcd))#removeme\nnumc = 8\nnumc = min(max(1, numc), h * w - 1)\ninds = totuple(asindices(c))\nlocs = sample(inds, numc)#removeme\nlocs = [(0, 2), (0, 4), (1, 2), (0, 0), (1, 4), (0, 3), (1, 0), (1, 1)]\nfgc = choice(cols)#removeme\nfgc = 6\ngi = fill(c, fgc, locs)\ngo = canvas(0, (h**2, w**2))\nfor loc in locs:\n go = fill(go, fgc, shift(locs, multiply(loc, (h, w))))\n\noutput[[6066600000606666066660666][6660600000666066660666606][6066660666606660000060666][6660666606666060000066606]]"
6,6,007bbfb7,"\npython\nfrom utils import *\nfrom dsl import *\nfrom random import seed as set_seed\nset_seed(466475820)\ncols = interval(1, 10, 1)\nh = unifint(diff_lb, diff_ub, (2, 5))#removeme\nh = 2\nw = unifint(diff_lb, diff_ub, (2, 5))#removeme\nw = 2\nc = canvas(0, (h, w))\nnumcd = unifint(diff_lb, diff_ub, (0, (h * w) // 2))#removeme\nnumcd = 2\nnumc = choice((numcd, h * w - numcd))#removeme\nnumc = 2\nnumc = min(max(1, numc), h * w - 1)\ninds = totuple(asindices(c))\nlocs = sample(inds, numc)#removeme\nlocs = [(0, 1), (1, 0)]\nfgc = choice(cols)#removeme\nfgc = 8\ngi = fill(c, fgc, locs)\ngo = canvas(0, (h**2, w**2))\nfor loc in locs:\n go = fill(go, fgc, shift(locs, multiply(loc, (h, w))))\n\noutput[[0008][0080][0800][8000]]"
11,11,007bbfb7,"\npython\nfrom utils import *\nfrom dsl import *\nfrom random import seed as set_seed\nset_seed(645980992)\ncols = interval(1, 10, 1)\nh = unifint(diff_lb, diff_ub, (2, 5))#removeme\nh = 5\nw = unifint(diff_lb, diff_ub, (2, 5))#removeme\nw = 2\nc = canvas(0, (h, w))\nnumcd = unifint(diff_lb, diff_ub, (0, (h * w) // 2))#removeme\nnumcd = 4\nnumc = choice((numcd, h * w - numcd))#removeme\nnumc = 6\nnumc = min(max(1, numc), h * w - 1)\ninds = totuple(asindices(c))\nlocs = sample(inds, numc)#removeme\nlocs = [(3, 1), (2, 1), (4, 0), (4, 1), (1, 0), (0, 1)]\nfgc = choice(cols)#removeme\nfgc = 5\ngi = fill(c, fgc, locs)\ngo = canvas(0, (h**2, w**2))\nfor loc in locs:\n go = fill(go, fgc, shift(locs, multiply(loc, (h, w))))\n\noutput[[0005][0050][0005][0005][0055][0500][5000][0500][0500][5500][0005][0050][0005][0005][0055][0005][0050][0005][0005][0055][0505][5050][0505][0505][5555]]"
16,16,00d62c1b,"\npython\nfrom utils import *\nfrom dsl import *\nfrom random import seed as set_seed\nset_seed(559356030)\ncols = remove(4, interval(0, 10, 1))\nh = unifint(diff_lb, diff_ub, (5, 30))#removeme\nh = 17\nw = unifint(diff_lb, diff_ub, (5, 30))#removeme\nw = 18\nbgc, fgc = sample(cols, 2)#removeme\nbgc, fgc = 8, 5\ngi = canvas(bgc, (h, w))\nnblocks = unifint(diff_lb, diff_ub, (1, (h * w) // 20))#removeme\nnblocks = 13\nsucc = 0\ntr = 0\nmaxtr = 5 * nblocks\ninds = asindices(gi)\nwhile succ < nblocks and tr < maxtr:\n tr += 1\n oh = randint(3, 8)\n ow = randint(3, 8)\n cands = sfilter(inds, lambda ij: ij[0] <= h - oh and ij[1] <= w - ow)\n if len(cands) == 0:\n continue\n loc = choice(totuple(cands))\n loci, locj = loc\n bx = box(frozenset({(loci, locj), (loci + oh - 1, locj + ow - 1)}))\n bx = bx - set(sample(totuple(corners(bx)), randint(0, 4)))\n if bx.issubset(inds) and len(inds - bx) > (h * w) // 2 + 1:\n gi = fill(gi, fgc, bx)\n ..."
18,18,00d62c1b,"\npython\nfrom utils import *\nfrom dsl import *\nfrom random import seed as set_seed\nset_seed(853199386)\ncols = remove(4, interval(0, 10, 1))\nh = unifint(diff_lb, diff_ub, (5, 30))#removeme\nh = 28\nw = unifint(diff_lb, diff_ub, (5, 30))#removeme\nw = 23\nbgc, fgc = sample(cols, 2)#removeme\nbgc, fgc = 5, 3\ngi = canvas(bgc, (h, w))\nnblocks = unifint(diff_lb, diff_ub, (1, (h * w) // 20))#removeme\nnblocks = 7\nsucc = 0\ntr = 0\nmaxtr = 5 * nblocks\ninds = asindices

In [29]:
print(df[df.id==1563].iloc[0].text)

input[[331133][311113][311113][313313][133331]]
python
x0 = double(TEN)
x1 = interval(ZERO, x0, ONE)
x2 = connect(ORIGIN, DOWN)
x3 = connect(ORIGIN, ZERO_BY_TWO)
x4 = combine(x2, x3)
x5 = astuple(NEG_TWO, TWO)
x6 = lbind(multiply, x5)
x7 = toivec(NEG_TWO)
x8 = apply(x6, x1)
x9 = rbind(subtract, TWO_BY_ZERO)
x10 = fork(ofcolor, identity, leastcolor)
x11 = chain(x9, center, x10)
x12 = rbind(mapply, x8)
x13 = lbind(lbind, shift)
x14 = lbind(shift, x4)
x15 = compose(x14, x11)
x16 = chain(x12, x13, x15)
x17 = lbind(recolor, FIVE)
x18 = compose(x17, x16)
x19 = fork(paint, identity, x18)
x20 = compose(rot180, x19)
x21 = fork(ofcolor, x20, leastcolor)
x22 = compose(center, x21)
x23 = fork(subtract, x22, x11)
x24 = fork(shift, x16, x23)
x25 = lbind(recolor, FIVE)
x26 = rbind(shift, x7)
x27 = chain(x25, x26, x24)
x28 = fork(paint, x20, x27)
x29 = compose(rot180, x28)
x30 = rbind(ofcolor, FIVE)
x31 = compose(x30, x29)
x32 = leastcolor(I)
x33 = ofcolor(I, x32)
x34 = mostcolor(I)
x35 = shape(I)
x36

In [20]:
df

,id,task,text
0,0,007bbfb7,"input[[4440444404444040000044404][4404444044440440000044044][4444444444444440000044444][0440404404044040000004404][4440444404000004440444404][4404444044000004404444044][4444444444000004444444444][0440404404000000440404404][4440444404444044440444404][4404444044440444404444044][4444444444444444444444444][0440404404044040440404404][0000044404444040000044404][0000044044440440000044044][0000044444444440000044444][0000004404044040000004404]]\npython\nx0 = objects(I, T, F, T)\nx1 = rbind(greater, TWO)\nx2 = chain(x1, minimum, shape)\nx3 = fork(equality, toindices, box)\nx4 = fork(both, x2, x3)\nx5 = mfilter(x0, x4)\nx6 = fill(I, THREE, x5)\n\noutput of x6\n[[4440444404444040000044404][4404444044440440000044044][4444444444444440000044444][0440404404044040000004404][4440444404000004440444404][4404444044000004404444044][4444444444000004444444444][0440404404000000440404404][4440444404444044440444404][4404444044440444404444044][4444444444444444444444444][0440404404044040440404404][000004440444..."
1,1,007bbfb7,"input[[6006][6660][6660]]\npython\nfrom utils import *\nfrom dsl import *\nfrom random import seed as set_seed\nset_seed(581116249)\ncols = interval(1, 10, 1)\nh = unifint(diff_lb, diff_ub, (2, 5))#removeme\nh = 3\nw = unifint(diff_lb, diff_ub, (2, 5))#removeme\nw = 4\nc = canvas(0, (h, w))\nnumcd = unifint(diff_lb, diff_ub, (0, (h * w) // 2))#removeme\nnumcd = 4\nnumc = choice((numcd, h * w - numcd))#removeme\nnumc = 8\nnumc = min(max(1, numc), h * w - 1)\ninds = totuple(asindices(c))\nlocs = sample(inds, numc)#removeme\nlocs = [(1, 1), (2, 0), (0, 0), (1, 2), (2, 1), (2, 2), (0, 3), (1, 0)]\nfgc = choice(cols)#removeme\nfgc = 6\ngi = fill(c, fgc, locs)\ngo = canvas(0, (h**2, w**2))\nfor loc in locs:\n go = fill(go, fgc, shift(locs, multiply(loc, (h, w))))\n\n"
2,2,007bbfb7,"\npython\nfrom utils import *\nfrom dsl import *\nfrom random import seed as set_seed\nset_seed(780605261)\ncols = interval(1, 10, 1)\nh = unifint(diff_lb, diff_ub, (2, 5))#removeme\nh = 2\nw = unifint(diff_lb, diff_ub, (2, 5))#removeme\nw = 5\nc = canvas(0, (h, w))\nnumcd = unifint(diff_lb, diff_ub, (0, (h * w) // 2))#removeme\nnumcd = 2\nnumc = choice((numcd, h * w - numcd))#removeme\nnumc = 8\nnumc = min(max(1, numc), h * w - 1)\ninds = totuple(asindices(c))\nlocs = sample(inds, numc)#removeme\nlocs = [(0, 2), (0, 4), (1, 2), (0, 0), (1, 4), (0, 3), (1, 0), (1, 1)]\nfgc = choice(cols)#removeme\nfgc = 6\ngi = fill(c, fgc, locs)\ngo = canvas(0, (h**2, w**2))\nfor loc in locs:\n go = fill(go, fgc, shift(locs, multiply(loc, (h, w))))\n\noutput[[6066600000606666066660666][6660600000666066660666606][6066660666606660000060666][6660666606666060000066606]]"
3,3,007bbfb7,"input[[6006000000006006][6660000000006660][6660000000006660][6006600660060000][6660666066600000][6660666066600000][6006600660060000][6660666066600000][6660666066600000]]\npython\nx0 = fgpartition(I)\nx1 = merge(x0)\nx2 = height(x1)\nx3 = height(I)\nx4 = interval(ZERO, x3, x2)\nx5 = lbind(shift, x1)\nx6 = compose(x5, toivec)\nx7 = compose(x6, invert)\nx8 = fork(combine, x6, x7)\nx9 = mapply(x8, x4)\nx10 = paint(I, x9)\n\noutput of x10\n[[6006000000006006][6660000000006660][6660000000006660][6006600660060000][6660666066600000][6660666066600000][6006600660060000][6660666066600000][6660666066600000]]"
4,4,007bbfb7,"input[[60666][66606]]\npython\nx0 = fgpartition(I)\nx1 = merge(x0)\nx2 = toindices(x1)\nx3 = box(x2)\nx4 = difference(x3, x2)\nx5 = inbox(x2)\nx6 = backdrop(x5)\nx7 = lbind(position, x6)\nx8 = compose(x7, initset)\nx9 = lowermost(x6)\nx10 = rightmost(x6)\nx11 = uppermost(x6)\nx12 = leftmost(x6)\nx13 = rbind(greater, x9)\nx14 = compose(x13, first)\nx15 = lbind(greater, x11)\nx16 = compose(x15, first)\nx17 = rbind(greater, x10)\nx18 = compose(x17, last)\nx19 = lbind(greater, x12)\nx20 = compose(x19, last)\nx21 = compose(invert, x16)\nx22 = fork(add, x14, x21)\nx23 = compose(invert, x20)\nx24 = fork(add, x18, x23)\nx25 = fork(astuple, x22, x24)\nx26 = fork(shoot, iden

In [7]:
df[df.text.str.contains('function')]

,id,task,text
11,11,007bbfb7,output of x6\n[[<function<unk>identity<unk>at<unk>0x7f4e1c54c9d0><function<unk>identity<unk>at<unk>0x7f4e1c54c9d0>][<function<unk>rot90<unk>at<unk>0x7f4e1c5948b0><function<unk>rot270<unk>at<unk>0x7f4e1c5949d0>][<function<unk>rot180<unk>at<unk>0x7f4e1c594940><function<unk>rot180<unk>at<unk>0x7f4e1c594940>][<function<unk>rot270<unk>at<unk>0x7f4e1c5949d0><function<unk>rot90<unk>at<unk>0x7f4e1c5948b0>]]
68,68,05f2a901,output of x13\n[[<function<unk>identity<unk>at<unk>0x7f4e1c54c9d0><function<unk>identity<unk>at<unk>0x7f4e1c54c9d0>][<function<unk>rot90<unk>at<unk>0x7f4e1c5948b0><function<unk>rot270<unk>at<unk>0x7f4e1c5949d0>][<function<unk>rot180<unk>at<unk>0x7f4e1c594940><function<unk>rot180<unk>at<unk>0x7f4e1c594940>][<function<unk>rot270<unk>at<unk>0x7f4e1c5949d0><function<unk>rot90<unk>at<unk>0x7f4e1c5948b0>]]
71,71,06df4c85,output of x22\n[[<function<unk>identity<unk>at<unk>0x7f4e1c54c9d0><function<unk>identity<unk>at<unk>0x7f4e1c54c9d0>][<function<unk>rot90<unk>at<unk>0x7f4e1c5948b0><function<unk>rot270<unk>at<unk>0x7f4e1c5949d0>][<function<unk>rot180<unk>at<unk>0x7f4e1c594940><function<unk>rot180<unk>at<unk>0x7f4e1c594940>][<function<unk>rot270<unk>at<unk>0x7f4e1c5949d0><function<unk>rot90<unk>at<unk>0x7f4e1c5948b0>]]
72,72,06df4c85,output of x5\n[<function<unk>identity<unk>at<unk>0x7f4e1c54c9d0><function<unk>dmirror<unk>at<unk>0x7f4e1c594b80><function<unk>cmirror<unk>at<unk>0x7f4e1c594c10><function<unk>hmirror<unk>at<unk>0x7f4e1c594a60>]
93,93,09629e4f,output of x2\n[<function<unk>identity<unk>at<unk>0x7f4e1c54c9d0><function<unk>dmirror<unk>at<unk>0x7f4e1c594b80><function<unk>cmirror<unk>at<unk>0x7f4e1c594c10><function<unk>hmirror<unk>at<unk>0x7f4e1c594a60>]
143,143,0dfd9992,output of x13\n[[<function<unk>identity<unk>at<unk>0x7f4e1c54c9d0><function<unk>identity<unk>at<unk>0x7f4e1c54c9d0>][<function<unk>rot90<unk>at<unk>0x7f4e1c5948b0><function<unk>rot270<unk>at<unk>0x7f4e1c5949d0>][<function<unk>rot180<unk>at<unk>0x7f4e1c594940><function<unk>rot180<unk>at<unk>0x7f4e1c594940>][<function<unk>rot270<unk>at<unk>0x7f4e1c5949d0><function<unk>rot90<unk>at<unk>0x7f4e1c5948b0>]]
168,168,11852cab,output of x22\n[[<function<unk>identity<unk>at<unk>0x7f4e1c54c9d0><function<unk>identity<unk>at<unk>0x7f4e1c54c9d0>][<function<unk>rot90<unk>at<unk>0x7f4e1c5948b0><function<unk>rot270<unk>at<unk>0x7f4e1c5949d0>][<function<unk>rot180<unk>at<unk>0x7f4e1c594940><function<unk>rot180<unk>at<unk>0x7f4e1c594940>][<function<unk>rot270<unk>at<unk>0x7f4e1c5949d0><function<unk>rot90<unk>at<unk>0x7f4e1c5948b0>]]
211,211,1b2d62fb,output of x3\nfrozenset[{<function<unk>rbind.<locals>.<lambda><unk>at<unk>0x7f4e19e45090>}]
324,324,2204b7a8,output of x22\n[[<function<unk>identity<unk>at<unk>0x7f4e1c54c9d0><function<unk>identity<unk>at<unk>0x7f4e1c54c9d0>][<function<unk>rot90<unk>at<unk>0x7f4e1c5948b0><function<unk>rot270<unk>at<unk>0x7f4e1c5949d0>][<function<unk>rot180<unk>at<unk>0x7f4e1c594940><function<unk>rot180<unk>at<unk>0x7f4e1c594940>][<function<unk>rot270<unk>at<unk>0x7f4e1c5949d0><function<unk>rot90<unk>at<unk>0x7f4e1c5948b0>]]
342,342,22233c11,output of x3\nfrozenset[{<function<unk>rbind.<locals>.<lambda><unk>at<unk>0x7f4e1a0d7370>}]


# program

## dsr_qw2d5_1d5b

In [ ]:



import util



MODELNAME="DSR_qw2d5_1d5b_d21"
DS="deepscaler"
DSCLS="Dataset"
VDSCLS="Dataset"
TDSCLS="Dataset"
BACKBONE="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit"
# RESTORE=""
# BACKBONE=sorted(glob(f"../data/{RESTORE}_KF0/checkpoint-*"), key=lambda x: int(x.split('-')[-1]))[-1]

DATASEED=90987655
SEED=DATASEED
KN=5
KFIDS="0"


!./dls1.sh  -model_name "{MODELNAME}" -ds "{DS}" -ds_cls "{DSCLS}" -val_ds_cls "{VDSCLS}" -test_ds_cls "{TDSCLS}" -kn {KN}  -kfids "{KFIDS}" -backbone "{BACKBONE}" \
-data_seed {DATASEED} -seed {SEED} -eval_delay 0 -epochs 1  -do_train  -dataloader_num_workers 4 -es 1 -verbose 4 -disable_tqdm -save_strategy steps  -evaluation_strategy steps -eval_steps 100 \
-save_total_limit 10 -lr 1e-5 -warmup_ratio 0.01 -weight_decay 0.01 -bs 4 -gas 1 -vbs 1 -torch_dtype bfloat16 -use_bf16 -optim paged_adamw_8bit -gradient_checkpointing -local \
-use_unsloth -use_4bit -use_lora  -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules  gate_proj up_proj down_proj q_proj o_proj k_proj v_proj -torch_empty_cache_steps 1 \
-max_model_len 10240 -max_prompt_length 1024 -max_completion_length 7300   -use_vllm -vllm_gpu_memory_utilization 0.15 -ppt dsr -num_generations 8 -len_penalty 0.5 -temperature 0.9 -nv





logfile:../data/logs/20250222125459.log
params:-model_name DSR_qw2d5_1d5b_d21 -ds deepscaler -ds_cls Dataset -val_ds_cls Dataset -test_ds_cls Dataset -kn 5 -kfids 0 -backbone unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit -data_seed 90987655 -seed 90987655 -eval_delay 0 -epochs 1 -do_train -dataloader_num_workers 4 -es 1 -verbose 4 -disable_tqdm -save_strategy steps -evaluation_strategy steps -eval_steps 100 -save_total_limit 10 -lr 1e-5 -warmup_ratio 0.01 -weight_decay 0.01 -bs 4 -gas 1 -vbs 1 -torch_dtype bfloat16 -use_bf16 -optim paged_adamw_8bit -gradient_checkpointing -local -use_unsloth -use_4bit -use_lora -lora_dropout 0 -lora_alpha 16 -lora_rank 32 -lora_modules gate_proj up_proj down_proj q_proj o_proj k_proj v_proj -torch_empty_cache_steps 1 -max_model_len 10240 -max_prompt_length 1024 -max_completion_length 7300 -use_vllm -vllm_gpu_memory_utilization 0.15 -ppt dsr -num_generations 8 -len_penalty 0.5 -temperature 0.9 -nv


# sub

In [11]:
def find_subregions(matrix):
    def dfs(x, y, value):
        if x < 0 or y < 0 or x >= len(matrix) or y >= len(matrix[0]):
            return
        if matrix[x][y] != value or visited[x][y]:
            return
        
        # 标记为已访问
        visited[x][y] = True
        subregion.append((x, y))
        
        # 探索相邻的四个方向
        dfs(x+1, y, value)
        dfs(x-1, y, value)
        dfs(x, y+1, value)
        dfs(x, y-1, value)

    # 初始化结果列表和访问标记矩阵
    results = []
    visited = [[False for _ in range(len(matrix[0]))] for _ in range(len(matrix))]

    # 遍历每个元素
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if not visited[i][j]:
                subregion = []
                dfs(i, j, matrix[i][j])
                if subregion:
                    results.append(subregion)

    return results

# 示例使用
matrix = [
    [1, 1, 0, 2],
    [1, 1, 0, 0],
    [3, 0, 0, 0]
]

subregions = find_subregions(matrix)
print("Subregions:")
for subregion in subregions:
    print(subregion)

Subregions:
[(0, 0), (1, 0), (1, 1), (0, 1)]
[(0, 2), (1, 2), (2, 2), (2, 3), (1, 3), (2, 1)]
[(0, 3)]
[(2, 0)]


In [16]:
def generate_subarrays(dimensions):
    """
    生成所有可能的子块（子矩阵）。

    参数:
    dimensions (tuple): 一个包含两个整数的元组 (rows, cols)，表示二维数组的行数和列数。

    返回:
    list: 包含所有可能子块的列表，每个子块是一个二维列表。
    """
    rows, cols = dimensions
    subarrays = []

    # 遍历所有可能的子块起点和终点
    for start_row in range(rows):
        for start_col in range(cols):
            for end_row in range(start_row, rows):
                for end_col in range(start_col, cols):
                    # 创建子块
                    subarray = [[0 for _ in range(end_col - start_col + 1)] for _ in range(end_row - start_row + 1)]
                    subarrays.append(subarray)

    return subarrays

# 示例使用
if __name__ == "__main__":
    dimensions = (3, 5)  # 定义二维数组的维度
    result = generate_subarrays(dimensions)
    print("所有可能的子块:")
    for idx, subarray in enumerate(result):
        print(f"子块 {idx + 1}:")
        for row in subarray:
            print(row)
        print()

所有可能的子块:
子块 1:
[0]

子块 2:
[0, 0]

子块 3:
[0, 0, 0]

子块 4:
[0, 0, 0, 0]

子块 5:
[0, 0, 0, 0, 0]

子块 6:
[0]
[0]

子块 7:
[0, 0]
[0, 0]

子块 8:
[0, 0, 0]
[0, 0, 0]

子块 9:
[0, 0, 0, 0]
[0, 0, 0, 0]

子块 10:
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]

子块 11:
[0]
[0]
[0]

子块 12:
[0, 0]
[0, 0]
[0, 0]

子块 13:
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

子块 14:
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]

子块 15:
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]

子块 16:
[0]

子块 17:
[0, 0]

子块 18:
[0, 0, 0]

子块 19:
[0, 0, 0, 0]

子块 20:
[0]
[0]

子块 21:
[0, 0]
[0, 0]

子块 22:
[0, 0, 0]
[0, 0, 0]

子块 23:
[0, 0, 0, 0]
[0, 0, 0, 0]

子块 24:
[0]
[0]
[0]

子块 25:
[0, 0]
[0, 0]
[0, 0]

子块 26:
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

子块 27:
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]

子块 28:
[0]

子块 29:
[0, 0]

子块 30:
[0, 0, 0]

子块 31:
[0]
[0]

子块 32:
[0, 0]
[0, 0]

子块 33:
[0, 0, 0]
[0, 0, 0]

子块 34:
[0]
[0]
[0]

子块 35:
[0, 0]
[0, 0]
[0, 0]

子块 36:
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

子块 37:
[0]

子块 38:
[0, 0]

子块 39:
[0]
[0]

子块 40:
[0, 0]
[0, 0]

子块 41:
[0]
[0]
[0

In [2]:
!curl http://192.168.0.104:33333/v1/completions \
-H "Content-Type: application/json" \
-d '{"model": "custom","prompt": "San Francisco is a","max_tokens": 7,"temperature": 0}'

{"id":"cmpl-d8d6603eb69b47188254d178a0f5cda5","object":"text_completion","created":1724320607,"model":"custom","choices":[{"index":0,"text":" city of contrasts. It's a","logprobs":null,"finish_reason":"length","stop_reason":null}],"usage":{"prompt_tokens":4,"total_tokens":11,"completion_tokens":7}}

# submission

In [10]:
sub = util.load_json('../data/submission.json')
sub_ice = util.load_json('../data/sub_icecube.json')

In [13]:
len(sub_ice)

100

In [14]:
n = 0
for k, vs in sub_ice.items():
    for i, v in enumerate(vs):
        if v['attempt_1'] !=v['attempt_2']:
            print(k, i, 1)
            n += 1
print(n)
       

3345333e 0 1
22168020 0 1
0b148d64 0 1
0520fde7 0 1
3428a4f5 0 1
3428a4f5 1 1
0a938d79 0 1
228f6490 0 1
045e512c 0 1
3af2c5a8 0 1
4093f84a 0 1
40853293 0 1
0ca9ddb6 0 1
29ec7d0e 0 1
1fad071e 0 1
2dd70a9a 0 1
1f0c79e5 0 1
23b5c85d 0 1
29623171 0 1
3de23699 0 1
2013d3e2 0 1
253bf280 0 1
2dc579da 0 1
1b60fb0c 0 1
05269061 0 1
3618c87e 0 1
3eda0437 0 1
25d8a9c8 0 1
39a8645d 0 1
05f2a901 0 1
178fcbfb 0 1
25ff71a9 0 1
25ff71a9 1 1
3bd67248 0 1
264363fd 0 1
1f85a75f 0 1
445eab21 0 1
42a50994 0 1
137eaa0f 0 1
3ac3eb23 0 1
1c786137 0 1
41e4d17e 0 1
1f642eb9 0 1
1f876c06 0 1
1bfc4729 0 1
363442ee 0 1
444801d8 0 1
22233c11 0 1
2c608aff 0 1
234bbc79 0 1
0dfd9992 0 1
28e73c20 0 1
3906de3d 0 1
28bf18c6 0 1
39e1d7f9 0 1
36d67576 0 1
239be575 0 1
239be575 1 1
3befdf3e 0 1
017c7c7b 0 1
007bbfb7 0 1
3aa6fb7a 0 1
3e980e27 0 1
1190e5a7 0 1
1cf80156 0 1
2bcee788 0 1
2204b7a8 0 1
2281f1f4 0 1
1caeab9d 0 1
272f95fa 0 1
150deff5 0 1
27a28665 0 1
27a28665 1 1
27a28665 2 1
1a07d186 0 1
3bdb4ada 0 1
1b2d62fb 0 1

# tokenzers

In [3]:
from transformers import AutoTokenizer, AutoConfig
modelid="BAAI/bge-small-en-v1.5"
tokenizer = AutoTokenizer.from_pretrained(modelid)
AutoConfig.from_pretrained(modelid)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

BertConfig {
  "_name_or_path": "BAAI/bge-small-en-v1.5",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [4]:
tokenizer.encode('ABAABBAAACCCC')

[101, 19557, 7875, 3676, 11057, 9468, 9468, 102]

In [29]:
tokenizer.convert_tokens_to_ids(["input"]+[str(x) for x in [0, 1, 1, 2, 2, 3]])

[4525, 32, 33, 33, 34, 34, 35]

In [22]:
tokenizer.encode(tokenizer.decode(tokenizer.convert_tokens_to_ids(["input:"]+[str(x) for x in [0, 1, 1, 2, 2, 3]])))

[0, 32, 33, 33, 34, 34, 35]

In [31]:
tokenizer.encode('input011223', add_special_tokens=False)

[4525, 32, 33, 33, 34, 34, 35]

In [30]:
tokenizer('input:')

{'input_ids': [4525, 42], 'attention_mask': [1, 1]}

In [50]:
tokenizer.eos_token

'<｜end▁of▁sentence｜>'

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-Instruct")

/home/tom/anaconda3/envs/hpp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
tokenizer.encode('what!\n\nThe answer is A')

[128000, 12840, 2268, 791, 4320, 374, 362]